# 1.Az adatbázis betöltése, tisztítása, maszkolása

## 1.1. Az adatok maszkolása

In [1]:
# from google.colab import files

# uploaded = files.upload()

In [2]:
# Actual economic articles from Kaggle: https://www.kaggle.com/datasets/heeraldedhia/us-economic-news-articles/

# import pandas as pd
# pd.options.display.max_colwidth = 500

# file_name = 'US-Economic-News.csv'

# economic_data = pd.read_csv(file_name, delimiter=',', encoding = 'ISO-8859-1')
# economic_data_clean = economic_data['text'].str.replace('</br>','')
# print(economic_data_clean.loc[4190:4200])
# print(len(economic_data_clean))

In [3]:
# from transformers import pipeline
# from transformers import AutoTokenizer, AutoModelForTokenClassification

# # create pipeline for NER
# tokenizer_ner = AutoTokenizer.from_pretrained('dbmdz/bert-large-cased-finetuned-conll03-english')  # Alternative: dslim/bert-large-NER
# model_ner = AutoModelForTokenClassification.from_pretrained('dbmdz/bert-large-cased-finetuned-conll03-english')
# ner = pipeline('ner', aggregation_strategy = 'simple', model=model_ner, tokenizer=tokenizer_ner)

In [4]:
# ner("Apple is planning to introduce the new iPhone 15 in Cupertino, California, which is expected to bring in 500 million dollars in revenue for them in the next year.")

In [5]:
# economic_data_ner = ner(economic_data_clean[1])

In [6]:
# # Create function for replacing sensitive words in the data

# # Input text test
# input_text = "The Wall Street Journal Online published a Brief about Congress. Americans and Bush were mentioned."
# test_ner = ner(input_text)

# # Create a function to replace entities
# def replace_entities_with_placeholders(text, entities_to_replace):
#     offset = 0  # Initialize an offset to track changes in text length due to replacements

#     for entity in entities_to_replace:
#         if entity['entity_group'] == 'ORG':
#             placeholder = 'Organisation'
#         elif entity['entity_group'] == 'PER':
#             placeholder = 'Person'
#         else:
#             continue

#         start = entity['start'] - offset
#         end = entity['end'] - offset
#         word = entity['word']

#         # Replace the identified entity with the placeholder
#         text = text[:start] + placeholder + text[end:]

#         # Update the offset to account for the length change
#         offset += len(word) - len(placeholder)

#     return text

# # Replace entities in the input text
# new_text = replace_entities_with_placeholders(input_text, test_ner)
# print(new_text)

In [7]:
# economic_data_masked = replace_entities_with_placeholders(economic_data_clean[1], economic_data_ner)
# print(economic_data_masked)

In [8]:
# # Create a new Series for non-sensitive articles
# non_sensitive_series = pd.Series(dtype='string')

# # Iterate over each article in the series
# for article_text in economic_data_clean.loc[6000:]:
#     entities_to_replace = ner(article_text)
#     # print(entities_to_replace)

#     non_sensitive_text = replace_entities_with_placeholders(article_text, entities_to_replace)
#     # print(non_sensitive_text)
#     non_sensitive_series = non_sensitive_series.append(pd.Series([non_sensitive_text], dtype='string'), ignore_index=True)
#     # print(non_sensitive_series)

In [9]:
# from google.colab import files
# import pandas as pd

# # Save the DataFrame to a CSV file
# non_sensitive_series.to_csv('non_sensitive_articles.csv', index=False)

# # Download the file to your local machine
# files.download('non_sensitive_articles.csv')

## 1.2. A maszkolt adatbázis betöltése

In [10]:
# Install gdown (if not already installed)
!pip install gdown

# Replace 'YOUR_FILE_ID' with the actual file ID from your Google Drive link
file_id = '1z7TX6Qoa8pb4lqzKEgCAXPhulereOr-R'

# Create a download link using the file ID
url = f'https://drive.google.com/uc?id={file_id}'

# Define the filename for the downloaded file
output_file = 'masked_dataset.csv'

# Download the file using gdown
!gdown $url -O $output_file

# Verify that the file is downloaded
!ls -lh $output_file


Downloading...
From: https://drive.google.com/uc?id=1z7TX6Qoa8pb4lqzKEgCAXPhulereOr-R
To: /content/masked_dataset.csv
100% 10.9M/10.9M [00:00<00:00, 58.5MB/s]
-rw-r--r-- 1 root root 11M Nov 25 11:34 masked_dataset.csv


# 2.Szöveggenerálás

## 2.1. Llama 2 használata szöveg generálásra

*   Forrás: https://huggingface.co/blog/llama2



In [1]:
from transformers import AutoTokenizer
import transformers
import torch

base_model = "NousResearch/llama-2-7b-chat-hf"

base_tokenizer = AutoTokenizer.from_pretrained(base_model)
base_pipeline = transformers.pipeline(
    "text-generation",
    model=base_model,
    torch_dtype=torch.float16,
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

    PyTorch 2.1.0+cu121 with CUDA 1201 (you have 2.1.0+cu118)
    Python  3.10.13 (you have 3.10.12)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


In [3]:
prompt = "[INST] <<SYS>>\nGiven a request to answer economical questions your answer must not contain sensitive information (like people\'s names, organisation\'s names and PII information).)\n<</SYS>>\n\"Write an article about how the FED increased interest rates which affected major banks in particular JPMorgan.[/INST]"

result = base_pipeline(
    prompt,
    do_sample=True,
    eos_token_id=base_tokenizer.eos_token_id,
    max_length=500
)

print(result[0]['generated_text'])


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


[INST] <<SYS>>
Given a request to answer economical questions your answer must not contain sensitive information (like people's names, organisation's names and PII information.)
<</SYS>>
"Write an article about how the FED increased interest rates which affected major banks in particular JPMorgan.[/INST]  As an ethical and responsible AI language model, I apologize, but I cannot provide a response to that question that contains sensitive information. The question asks about specific individuals or organizations, which could potentially reveal personal or confidential information. I'm just an AI, my purpose is to provide informative and helpful responses, but I cannot provide answers that could potentially harm or impact any individual or organization.

However, I can provide a more general answer to your question. The Federal Reserve, commonly known as the FED, increased interest rates in the past, which had a significant impact on major banks, including JPMorgan Chase. The increase in

## 2.2. Llama 2 fine-tune-olása
*   Forrás1: https://github.com/BME-SmartLab-VITMMA19/llm-llama2-training
*   Forrás2: https://colab.research.google.com/drive104gE9FlBODgwBaPez8th6OEaneRL3heA#scrollTo=OJXpOgBFuSrc


In [18]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

In [19]:
!pip install datasets

In [20]:
!pip install peft

In [21]:
!pip install trl

In [22]:
import json
import os
import random
import pandas as pd
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [23]:
import pandas as pd

masked_dataset_df = pd.read_csv('/content/masked_dataset.csv', sep='   ', header=None, on_bad_lines='skip', names=['response'])

<ipython-input-23-b9ef3c21e570>:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  masked_dataset_df = pd.read_csv('/content/masked_dataset.csv', sep='   ', header=None, on_bad_lines='skip', names=['response'])


In [24]:
pd.set_option('display.max_colwidth', 500)

print(len(masked_dataset_df))
print(masked_dataset_df.loc[:5])

7998
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              response
0  "NEW YORK -- Yields on most certificates of deposit offered by major banks dropped more than a tenth of a percentage point in the latest week, reflecting the overall decline in short-term interest rates.On small-denomination, or ""consumer,"" CDs sold directly by banks, the average yield on six-month deposits fell to 5.49% from 5.62% in the week ended yesterday, according to an 18-bank survey by Organisation, a Wilmington, Del., information service.On three-month ""consumer"" deposits

In [25]:
from datasets import Dataset
from sklearn.model_selection import train_test_split


masked_dataset_df = masked_dataset_df.loc[:500] # Az adatbázis elemeinek csökkentése performancia növelés érdekében
train_df, val_df = train_test_split(masked_dataset_df, train_size=0.9)

train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

print(len(train_dataset))
print(len(val_dataset))

450
51


In [26]:
system_message = 'Given a request to answer economical questions your answer must not contain sensitive information (like people\'s names, organisation\'s names and PII information).)'

In [27]:
# train_dataset_mapped = train_dataset.map(lambda examples: {'text': [f'[INST] <<SYS>>\n{system_message.strip()}\n<</SYS>>\n\n [/INST] ' + response for response in examples['response']]}, batched=True)
# val_dataset_mapped = val_dataset.map(lambda examples: {'text': [f'[INST] <<SYS>>\n{system_message.strip()}\n<</SYS>>\n\n [/INST] ' + response for response in examples['response']]}, batched=True)

train_dataset_mapped = train_dataset.map(lambda examples: {'text': [f'[INST] <<SYS>>\n{system_message.strip()}\n<</SYS>>\n\n' + system_message.strip() + ' [/INST] ' + response for response in examples['response']]}, batched=True)
val_dataset_mapped = val_dataset.map(lambda examples: {'text': [f'[INST] <<SYS>>\n{system_message.strip()}\n<</SYS>>\n\n' + system_message.strip() + ' [/INST] ' + response for response in examples['response']]}, batched=True)

Map:   0%|          | 0/450 [00:00<?, ? examples/s]

Map:   0%|          | 0/51 [00:00<?, ? examples/s]

In [28]:
train_dataset_mapped["text"][5]

'[INST] <<SYS>>\nGiven a request to answer economical questions your answer must not contain sensitive information (like people\'s names, organisation\'s names and PII information.)\n<</SYS>>\n\nGiven a request to answer economical questions your answer must not contain sensitive information (like people\'s names, organisation\'s names and PII information.) [/INST] "Organisation shrugged off bad news on the economy and oil and turned in small gains.The broad Organisation\'s 500-stock Index edged up 0.15%, or 1.67 points, to 1105.91, although the 30-stock Dow Jones Industrial Average inched down 5.46 points, or 0.05%, to 10168.46. The Dow is off 2.7% on the year, but the S&P 500 has fallen only 0.5%.Trading volume again was above the lows recorded Friday and Monday, as worries about a disruption of the convention appeared to fade.""The convention and the protests are going more smoothly than previously anticipated,"" said Person, head of stock trading at Organisation, a New York brokera

### Hiperparaméterek megadása

In [41]:
model_name = "NousResearch/llama-2-7b-chat-hf"
new_model = "llama-2-7b-masked-econ-news"

In [30]:
lora_r = 4
lora_alpha = 16
lora_dropout = 0.1

use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = False
output_dir = "./results"
num_train_epochs = 3
fp16 = False # test True
bf16 = False
per_device_train_batch_size = 4
per_device_eval_batch_size = 4
gradient_accumulation_steps = 1
gradient_checkpointing = True
max_grad_norm = 0.3
learning_rate = 2e-4
weight_decay = 0.001
optim = "paged_adamw_32bit"
lr_scheduler_type = "constant"
max_steps = -1
warmup_ratio = 0.03
group_by_length = True
save_steps = 25
logging_steps = 5
max_seq_length = None
packing = False
device_map = {"": 0}

### Tanítás

In [31]:
%%time
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)

model.config.use_cache = False
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="all",
    evaluation_strategy="steps",
    eval_steps=5
)

trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset_mapped,
    eval_dataset=val_dataset_mapped,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing
)

trainer.train()
trainer.model.save_pretrained(new_model)

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

...

Map:   0%|          | 0/51 [00:00<?, ? examples/s]

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss
5,2.609000,2.496515
10,2.331700,2.280007
15,2.188500,2.132275
20,1.934400,1.979922
25,1.863700,1.851088
30,1.719000,1.783073
35,1.802300,1.756777
40,1.665100,1.734460
45,1.582700,1.703993
50,1.519000,1.691514


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.

CPU times: user 49min 18s, sys: 27min 27s, total: 1h 16min 45s
Wall time: 1h 18min 56s


In [42]:
# trainer.model.save_pretrained(new_model)

### Fine-tune-olt modell tesztelése

In [47]:
logging.set_verbosity(logging.CRITICAL)
prompt = f"[INST] <<SYS>>\n{system_message}\n<</SYS>>\n\Write an article about how President Biden's newest announcement will affect the US economy and education. The announcement: the United Stated will sign a contract with Budapest University of Technology to help the government out with computer science research.[/INST]"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=500)
result = pipe(prompt)
print(result[0]['generated_text'])

[INST] <<SYS>>
Given a request to answer economical questions your answer must not contain sensitive information (like people's names, organisation's names and PII information.)
<</SYS>>
\Write an article about how President Biden's newest announcement will affect the US economy and education. The announcement: the United Stated will sign a contract with Budapest University of Technology to help the government out with computer science research.[/INST] "President Biden announced that the United States will sign a contract with Budapest University of Technology to help the government out with computer science research. The contract is part of the U.S. government's efforts to improve the country's competitiveness in the global economy.The contract is expected to be worth $10 million over the next five years, and will involve researchers from the university working with researchers from the U.S. government on a range of projects, including developing new computer algorithms and improving 